# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Prompt Engineering Best Practices: Building An End to End Customer Service System </center>

***


Prompt engineering plays a pivotal role in crafting queries that help large language models (LLMs) understand not just the language but also the nuance and intent behind the query and help us build complex applications with ease. 

In this notebook, we will put into action what we covered in previous notebooks and build an end-to-end customer service assistant. Starting with checking  the input to see if it flags the moderation API then extracting the list of products searching for the products the user asked about answering the user question with the model and checking the output with the moderation API. 

Finally, we will put all of these together and build a conversational chatbot the takes the user input passes it through all of these steps, and returns it back to him.

#### <a id="top"></a>
# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Setting Up Working Environment </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Chain of Prompts For Processing the User Query </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3. Building Conversational Chatbot </a></li> 
</ul>
</div>

***

<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Setting Up Working Environment </b></div>



As usual, we will start with setting up the working environment and importing the packages and libraries we will work on within this notebook. In addition to the usual packages such as os and openai, we will import the panel package which is a Python package we'll use for a chatbot UI. Also, we will import the utils file which will have some helper function that can be used to define the products and more.

In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
from openai import OpenAI
import sys
sys.path.append('/kaggle/input/chatdata')
import utils
import panel as pn  # GUI
pn.extension()

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)


Next, we will define the get_completion_from_messages which will take the messages with different types and return the LLM response.

In [3]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content


<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Chain of Prompts For Processing the User Query </b></div>



### The first step will check input to see if it flags the Moderation API or is a prompt injection:

```python
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
```

### The second step will be to extract the list of products

```python
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)
    
    if debug: print("Step 2: Extracted list of products.")
```

### The third step will be to check if these products are available we will look up the product information

```python
    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")
 ```        
 
### The fourth step will be to define the system message and define the messages that will be sent to the LLM to answer the user question

```python
    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]
```

### The fifth step will be to put the LLM response through the moderation method to make sure that it is free of harmful content

```python

    # Step 5: Put the answer through the Moderation API
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")
 ```

### Step 6 we will validate the LLM answer and make sure it follows the instruction and answer the user's question

```python

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")
 ```

### The final step is to return the answer to the user if it meets the previous conditions  

```python

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages
    
 ```
    

Let's put everything together and try it with different user messages to see how it will act

In [4]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
        
    response = client.moderations.create(input=user_input)
    moderation_output = response.results[0]


    if moderation_output.flagged:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    #print(print(category_and_product_response)
    
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API    
    response = client.moderations.create(input=final_response)
    moderation_output = response.results[0]
    

    if moderation_output.flagged:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

So we have the user input that we've been using. "**Tell me about the SmartX Pro phone and the camera. Also, tell me about TVs**." So let's run this.

In [5]:
user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Error: Invalid JSON string
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
- The SmartX Pro phone is a high-end smartphone with a powerful processor, advanced camera features, and a sleek design. It offers a large display, long battery life, and fast performance for multitasking and gaming.
- The FotoSnap camera is a DSLR camera with a high-resolution sensor, interchangeable lenses, and manual controls for professional photography. It offers features like optical zoom, image stabilization, and 4K video recording.
- Our range of TVs includes various sizes, resolutions (HD, Full HD, 4K), and smart TV capabilities. They offer features like HDR for enhanced picture quality, built-in streaming apps, and voice control options.

Do you have any specific questions about

As you can see, we're going through the steps to answer the user question. The first step is the moderation step, and the second step is extracting the list of products. The third step is looking up the product information. With this product information, the model is trying to answer the question. Finally, it puts the response through the moderation API again to make sure it's safe to show to the user.
Let's try another example but this time we will ask about a product that is not on the product list

In [6]:
user_input = "tell me about the Iphone 15"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
The iPhone 15 has not been released yet. Are you interested in learning more about the latest iPhone model currently available, such as the iPhone 13?


Let's try another prompt in which I will ask about a product that is out of the category list included in the product list

In [7]:
user_input = "tell me about the Harry Potter novel series"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
The Harry Potter novel series, written by J.K. Rowling, consists of seven books that follow the magical adventures of a young wizard named Harry Potter and his friends at Hogwarts School of Witchcraft and Wizardry. The series is beloved by readers of all ages and has inspired movies, merchandise, and a dedicated fan base. Are you looking to purchase the books or learn more about the series?


Although this product is not on the list the LLM managed to answer the query as they are a well-known product. Let's finally try a question about a product that does not exist and see how the final response will be

In [8]:
user_input = "tell me about the Xtg Fridge and what are the current available models"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
The Xtg Fridge is a popular model known for its energy efficiency and spacious design. It comes in various sizes and configurations to suit different needs. 

To provide you with the most accurate information on the current available models, could you please specify the size or specific features you are looking for in a fridge? This way, I can assist you in finding the perfect Xtg Fridge model for your requirements.


There is no Xtg Fridge and it is not the product list and this due to the LLM hallucination and you can solve this by editing the prompt.

<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Building Conversational Chatbot </b></div>



In the previous section, I introduced the process_user_message. It takes in the user input, which is the current message, and an array of all of the messages so far. So here we have a function that will just accumulate the messages as we interact with the assistant.

In [9]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

Finally, we will build a simple UI that will take the user messages and pass them to the collect_messages function return the response, and show it to the user.

In [10]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)

# <div style="box-shadow: rgba(240, 46, 170, 0.4) -5px 5px inset, rgba(240, 46, 170, 0.3) -10px 10px inset, rgba(240, 46, 170, 0.2) -15px 15px inset, rgba(240, 46, 170, 0.1) -20px 20px inset, rgba(240, 46, 170, 0.05) -25px 25px inset; padding:20px; font-size:30px; font-family: consolas; display:fill; border-radius:15px; color: rgba(240, 46, 170, 0.7)"> <b> ༼⁠ ⁠つ⁠ ⁠◕⁠‿⁠◕⁠ ⁠༽⁠つ Thank You!</b></div>

<p style="font-family:verdana; color:rgb(34, 34, 34); font-family: consolas; font-size: 16px;"> 💌 Thank you for taking the time to read through my notebook. I hope you found it interesting and informative. If you have any feedback or suggestions for improvement, please don't hesitate to let me know in the comments. <br><br> 🚀 If you liked this notebook, please consider upvoting it so that others can discover it too. Your support means a lot to me, and it helps to motivate me to create more content in the future. <br><br> ❤️ Once again, thank you for your support, and I hope to see you again soon!</p>